In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline
%load_ext autoreload
%autoreload 2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [ ]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np
import torch
import json
from pathlib import Path
from datetime import timedelta
from tqdm.auto import tqdm
import pickle

In [3]:
# Initialize CAVE client
datastack_name = "minnie65_phase3_v1"
my_token = "64df9664ce8a28852ee99167c26a9e8d"
client = CAVEclient(datastack_name=datastack_name, auth_token=my_token)
mat_version = 943
save_directory = f"../../data/test_minnie_dicts_{mat_version}"
# save_directory = f"../../data/minnie_dicts_{mat_version}"

In [4]:
# [date, timestamp, mean_coord_x, mean_coord_y, mean_coord_z, operation_id]
df = pd.read_feather('../../data/pruned_minnie_splitlog_943/splitlog_943.feather')
# [operation_id, sink supervoxel id, source_supervoxel id]
edits = np.load('../../data/pruned_minnie_splitlog_943/splitlog_943.npy')

In [6]:
# For testing purposes we are going to make edits a lot smaller
pruned_df = df.drop(df[df['operation_id'] > 105900].index)
test_edits = edits[:2500]

In [7]:
pruned_df.tail(5)

,date,timestamp,mean_coord_x,mean_coord_y,mean_coord_z,operation_id
0,2020-07-29 09:29:47.046010+00:00,1.596015e+09,117787.0,100298.5,18901.0,47222
1,2020-08-17 08:09:30.213000+00:00,1.597652e+09,122348.5,95767.5,21351.0,105222


In [8]:
print(test_edits[-5:])

[[            115795 111184679062833756 111184679062840384]
 [            115795 111184679062835594 111184679062840384]
 [            115795 111184679062836523 111184679062837823]
 [            115795 111184679062836523 111184679062837828]
 [            115795 111184679062837249 111184679062837823]]


In [21]:
import data_utils
op_to_svs = data_utils.create_operation_to_svs(client, mat_version, test_edits, save_directory, save=True)
print(len(op_to_svs))

49


In [24]:
op_to_pre_edit_dates = data_utils.create_operation_to_pre_edit_dates(mat_version, pruned_df, save_directory, save=True)
print(len(op_to_pre_edit_dates))
print(op_to_pre_edit_dates)

2
{47222: Timestamp('2020-07-29 09:29:47.045010'), 105222: Timestamp('2020-08-17 08:09:30.212000')}


In [25]:
num_processes = 2
op_to_pre_edit_roots = data_utils.create_operation_to_pre_edit_roots(client, mat_version, op_to_pre_edit_dates, op_to_svs, num_processes, save_directory, save=True)
print(len(op_to_pre_edit_roots))

  0%|          | 0/2 [00:00<?, ?it/s]


AttributeError: Can't pickle local object 'create_operation_to_pre_edit_roots.<locals>.process_operation'

In [ ]:
# Below is scratch

In [5]:
with open(f"{save_directory}/operation_to_pre_edit_dates_{mat_version}.pkl", 'rb') as f:
    op_to_pre_date = pickle.load(f)
print(len(op_to_pre_date))

496287


In [6]:
with open(f"{save_directory}/operation_to_supervoxels_{mat_version}.pkl", 'rb') as f:
    op_to_svs = pickle.load(f)
print(len(op_to_svs))

496287


In [7]:
ops = np.array(list(op_to_pre_date.keys()))
args_list = list([(ops[i], op_to_pre_date[ops[i]], op_to_svs[ops[i]].pop(), client) for i in range(len(ops))])
np_args_list = np.asarray(args_list, dtype="object")

In [8]:
import multiprocessing
from tqdm import tqdm


def fetch_root_id(client, supervoxel, date):
    # Call the client's polling function
    root_id = client.chunkedgraph.get_root_id(supervoxel, date)
    return root_id

def process_operation(data):
    op_id, date, supervoxel, client = data
    root_id = fetch_root_id(client, supervoxel, date)
    return op_id, root_id 

manager = multiprocessing.Manager()
op_to_root_ids = manager.dict()
with multiprocessing.Pool(processes=128) as pool, tqdm(total=len(ops)) as pbar:
    for op_id, root_id in pool.imap_unordered(process_operation, np_args_list):
        op_to_root_ids[op_id] = root_id
        pbar.update()

100%|██████████| 496287/496287 [1:02:31<00:00, 132.30it/s]


In [10]:
print(len(op_to_root_ids))

496287


In [11]:
with open(f"{save_directory}/operation_to_pre_edit_roots_{mat_version}.pkl", 'wb') as f:
    pickle.dump(op_to_root_ids, f)

In [12]:
with open(f"{save_directory}/operation_to_pre_edit_roots_{mat_version}.pkl", 'rb') as f:
    testing = pickle.load(f)
print(len(testing))

496287


In [57]:
cpus = len(os.sched_getaffinity(0))
print("cpus:", cpus)
clients = [CAVEclient(datastack_name=datastack_name, auth_token=my_token) for i in range(cpus)]
print(clients)

cpus: 16
[CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-

In [4]:
# [date, timestamp, mean_coord_x, mean_coord_y, mean_coord_z, operation_id]
df = pd.read_feather('../../data/pruned_minnie_splitlog_943/splitlog_943.feather')
# [operation_id, sink supervoxel id, source_supervoxel id]
edits = np.load('../../data/pruned_minnie_splitlog_943/splitlog_943.npy')

In [8]:
df.head(10)

,date,timestamp,mean_coord_x,mean_coord_y,mean_coord_z,operation_id
0,2020-07-29 09:29:47.046010+00:00,1.596015e+09,117787.000000,100298.500000,18901.000000,47222
1,2020-08-17 08:09:30.213000+00:00,1.597652e+09,122348.500000,95767.500000,21351.000000,105222
2,2020-08-20 14:21:23.740000+00:00,1.597933e+09,122647.500000,95861.000000,21358.000000,107815
3,2020-09-02 20:09:30.303000+00:00,1.599077e+09,162144.200000,46332.355556,26524.933333,115686
4,2020-09-02 20:09:50.511000+00:00,1.599077e+09,159585.500000,53807.000000,27261.000000,115687
5,2020-09-02 20:12:33.852000+00:00,1.599078e+09,159564.083333,53933.916667,27265.666667,115689
6,2020-09-02 20:12:37.750000+00:00,1.599078e+09,162263.002404,44109.615385,27237.492788,115690
7,2020-09-02 20:12:45.691000+00:00,1.599078e+09,147767.000000,119969.500000,23870.000000,115691
8,2020-09-02 20:14:15.103000+00:00,1.599078e+09,157509.000000,115241.500000,26416.000000,115700
9,2020-09-02 20:14:28.474000+00:00,1.599078e+09,152675.875000,121734.208333,23824.458333,115703


In [9]:
df.tail(10)

,date,timestamp,mean_coord_x,mean_coord_y,mean_coord_z,operation_id
496277,2024-01-21 16:02:43.092000+00:00,1.705853e+09,125098.633333,108029.577778,26699.00,903967
496278,2024-01-21 16:10:44.107000+00:00,1.705853e+09,106958.750000,87577.250000,19572.00,903969
496279,2024-01-21 16:11:07.639000+00:00,1.705853e+09,106821.000000,87684.666667,19573.00,903970
496280,2024-01-21 16:11:30.140000+00:00,1.705853e+09,106862.600000,87801.900000,19574.50,903971
496281,2024-01-21 16:12:25.789000+00:00,1.705854e+09,128587.750000,95285.000000,20869.00,903975
496282,2024-01-21 16:14:40.675000+00:00,1.705854e+09,128785.500000,105198.500000,19279.00,903977
496283,2024-01-21 16:16:16.363000+00:00,1.705854e+09,122913.000000,101388.833333,23303.50,903978
496284,2024-01-21 16:17:36.308000+00:00,1.705854e+09,122908.250000,106068.750000,20483.25,903980
496285,2024-01-21 16:18:12.197000+00:00,1.705854e+09,125220.000000,117978.250000,21464.50,903981
496286,2024-01-22 07:53:26.939000+00:00,1.705910e+09,98713.750000,90413.000000,21808.75,903984


In [58]:
from collections import defaultdict
op_to_svs = defaultdict(set)
for edit in edits:
    op_to_svs[edit[0]].add(edit[1])
    op_to_svs[edit[0]].add(edit[2])

In [59]:
print(len(op_to_svs))

496287


In [35]:
for item in op_to_svs:
    print(type(item))
    print(type(op_to_svs[item]))
    break

<class 'numpy.uint64'>
<class 'set'>


In [ ]:
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
with open(f"{save_directory}/operation_to_supervoxels_{mat_version}.pkl", 'wb') as f:
    pickle.dump(op_to_svs, f)

In [13]:
# nvm this is dumb
one = 0
mult = 0
for op_id in op_to_svs:
    if len(op_to_svs[op_id]) > 1:
        mult += 1
    else:
        one += 1
print("one", one)
print("mult", mult)

one 0
mult 496287


In [24]:
date = pd.to_datetime(df.loc[df['operation_id'] == 47222, 'date'].values[0])
pre_edit_date = date - one_ms
print(op_to_svs[op_id])
first = client.chunkedgraph.get_root_id(97197996596851211, pre_edit_date)
print(first)
print(op_to_svs[op_id])
pre_edit_root_ids = set(client.chunkedgraph.get_roots(list(op_to_svs[op_id]), pre_edit_date))
print(pre_edit_root_ids)

{97197996596851211, 97197996596845891, 97197996596845829}
864691135776571232
{97197996596851211, 97197996596845891, 97197996596845829}
{864691135776571232}


In [62]:
op_to_pre_date = {}
one_ms = timedelta(milliseconds=1)
for op_id in tqdm(df['operation_id']):
    date = pd.to_datetime(df.loc[df['operation_id'] == op_id, 'date'].values[0])
    pre_edit_date = date - one_ms
    op_to_pre_date[op_id] = pre_edit_date

100%|██████████| 496287/496287 [06:35<00:00, 1254.68it/s]


In [63]:
print(len(op_to_pre_date))

496287


In [84]:
with open(f"{save_directory}/operation_to_pre_edit_dates_{mat_version}.pkl", 'wb') as f:
    pickle.dump(op_to_pre_date, f)

In [96]:
import multiprocessing
from tqdm import tqdm
# print("cpu count", multiprocessing.cpu_count())
# print("cpu available", len(os.sched_getaffinity(0)))

def __process_op__(op_id, op_to_svs, op_to_pre_date, op_to_pre_edit_roots, lock):
        pre_edit_root_ids = client.chunkedgraph.get_root_id(op_to_svs[op_id].pop(), op_to_pre_date[op_id])
        #with lock:
        op_to_pre_edit_roots[op_id] = pre_edit_root_ids


manager = multiprocessing.Manager()
op_to_pre_edit_roots = manager.dict()
lock = multiprocessing.Lock()

ops = np.array(list(op_to_pre_date.keys()))[:50]
processes = []
with tqdm(total=len(ops)) as pbar:
    for i in range(len(ops)):
        p = multiprocessing.Process(target=__process_op__, args=(ops[i], op_to_svs, op_to_pre_date, op_to_pre_edit_roots, lock))
        processes.append(p)
        p.start()
        pbar.update()
        pbar.refresh()

    for p in processes:
        p.join()
pbar.close()

print(op_to_pre_edit_roots)

100%|██████████| 50/50 [00:04<00:00, 11.98it/s]

{47222: 864691135776571232, 105222: 864691136175092486, 107815: 864691136272969918, 115686: 864691134908636778, 115687: 864691135657862274, 115689: 864691136041121622, 115690: 864691135753780429, 115691: 864691136771677806, 115700: 864691135461774532, 115703: 864691135644727023, 115708: 864691136286652099, 115711: 864691135508790281, 115712: 864691135898139032, 115713: 864691135415525306, 115714: 864691135986944515, 115715: 864691136041121878, 115717: 864691136286652355, 115720: 864691136571690914, 115723: 864691136010985123, 115727: 864691135012579574, 115729: 864691135343693241, 115732: 864691135256264879, 115734: 864691136328938474, 115737: 864691135937424949, 115740: 864691135986944771, 115741: 864691136899949422, 115742: 864691135458893125, 115744: 864691136388307575, 115745: 864691135964657098, 115746: 864691135775830189, 115748: 864691136273045182, 115749: 864691135341090757, 115752: 864691135341091013, 115753: 864691136194122966, 115754: 864691136617175003, 115755: 864691136361

In [ ]:
import multiprocessing
from tqdm import tqdm
# print("cpu count", multiprocessing.cpu_count())
# print("cpu available", len(os.sched_getaffinity(0)))

def __process_op__(op_id, op_to_svs, op_to_pre_date, op_to_pre_edit_roots, lock):
        pre_edit_root_ids = client.chunkedgraph.get_root_id(op_to_svs[op_id].pop(), op_to_pre_date[op_id])
        #with lock:
        op_to_pre_edit_roots[op_id] = pre_edit_root_ids


manager = multiprocessing.Manager()
op_to_pre_edit_roots = manager.dict()
lock = multiprocessing.Lock()

ops = np.array(list(op_to_pre_date.keys()))[:50]
processes = []
with tqdm(total=len(ops)) as pbar:
    for i in range(len(ops)):
        p = multiprocessing.Process(target=__process_op__, args=(ops[i], op_to_svs, op_to_pre_date, op_to_pre_edit_roots, lock))
        processes.append(p)
        p.start()
        pbar.update()
        pbar.refresh()

    for p in processes:
        p.join()
pbar.close()

print(op_to_pre_edit_roots)

In [10]:
import multiprocessing
from tqdm import tqdm
import numpy as np
# print("cpu count", multiprocessing.cpu_count())
# print("cpu available", len(os.sched_getaffinity(0)))

def __process_op__(data):
    op_id, op_to_svs, op_to_pre_date, client = data
    ts_mat = client.materialize.get_timestamp(version=943)
    pre_edit_root_ids = client.chunkedgraph.get_root_id(97197996596845829, ts_mat)
    # pre_edit_root_ids = client.chunkedgraph.get_root_id(op_to_svs[op_id].pop(), op_to_pre_date[op_id])
    return pre_edit_root_ids

manager = multiprocessing.Manager()
op_to_pre_edit_roots = manager.dict()
op_to_pre_date_2 = manager.dict(op_to_pre_date)
op_to_svs_2 = manager.dict(op_to_svs)
# op_to_pre_edit_roots = {}
ops = np.array(list(op_to_pre_date.keys()))[:50]
args_list = [(ops[i], op_to_pre_date_2, op_to_svs_2, client) for i in range(len(ops))]
with multiprocessing.Pool() as p, tqdm(total=len(ops)) as pbar:
    for index, pre_edit_root_ids in enumerate(p.imap(__process_op__, args_list)):
        op_to_pre_edit_roots[ops[index]] = pre_edit_root_ids
        pbar.update()
        pbar.refresh()

KeyboardInterrupt: 

In [8]:
ops = np.array(list(op_to_pre_date.keys()))
args_list = [(op_id, op_to_pre_date[op_id], op_to_svs[op_id].pop(), client) for op_id in ops]
print(args_list[:5])

[(47222, Timestamp('2020-07-29 09:29:47.045010'), 97197996596845891, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>), (105222, Timestamp('2020-08-17 08:09:30.212000'), 98393028632728326, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>), (107815, Timestamp('2020-08-20 14:21:23.739000'), 98533834840494983, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>), (115686, Timestamp('2020-09-02 20:09:30.302000'), 109357290536546693, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>), (115687, Timestamp('2020-09-02 20:09:50.510000'), 108655596093501888, CAVEclient<datastack_name=minnie65_phase3_v1, server_address=https://global.daf-apis.com>)]


In [12]:
import multiprocessing
from tqdm import tqdm
# print("cpu count", multiprocessing.cpu_count())
# print("cpu available", len(os.sched_getaffinity(0)))

def __process_op__(op_id):
    pre_edit_root_ids = client.chunkedgraph.get_root_id(op_to_svs[op_id].pop(), op_to_pre_date[op_id])
    op_to_pre_edit_roots[op_id] = pre_edit_root_ids

op_to_pre_edit_roots = {}

ops = np.array(list(op_to_pre_date.keys()))
with multiprocessing.Pool() as p, tqdm(total=len(ops)) as pbar:
    for result in p.imap(__process_op__, ops):
        pbar.update()
        pbar.refresh()

  0%|          | 311/496287 [00:02<1:17:57, 106.03it/s]


KeyboardInterrupt: 

In [82]:
with open(f"{save_directory}/operation_to_pre_edit_roots_{mat_version}.pkl", 'wb') as f:
    pickle.dump(op_to_pre_edit_roots, f)

In [83]:
len(op_to_pre_edit_roots)

0

In [67]:
op_to_pre_edit_roots = {}
one_ms = timedelta(milliseconds=1)
for op_id in tqdm(op_to_pre_date.keys()):
    pre_edit_root_ids = set(client.chunkedgraph.get_roots(list(op_to_svs[op_id]), op_to_pre_date[op_id]))
    op_to_pre_edit_roots[op_id] = pre_edit_root_ids

  0%|          | 1818/496287 [04:05<18:30:59,  7.42it/s]


KeyboardInterrupt: 